In [60]:
import requests
import json
import time
import pandas as pd
import mysql.connector
from datetime import datetime
from config import api_football_key, conn_host, conn_database, conn_user, conn_password

In [61]:
league_id = 71 # BR League
season = 2022

now = time.time()

In [41]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

In [42]:
# Table creation queries
execute_query("CREATE TABLE IF NOT EXISTS leagues (id INT NOT NULL, name VARCHAR(50) NOT NULL, PRIMARY KEY (id))", False)
execute_query("CREATE TABLE IF NOT EXISTS teams (id INT NOT NULL, name VARCHAR(50) NOT NULL, PRIMARY KEY (id))", False)
execute_query("CREATE TABLE IF NOT EXISTS matches (id INT NOT NULL, date DATETIME NOT NULL, league_id INT NOT NULL, home_id INT NOT NULL, away_id INT NOT NULL, home_score INT NULL, away_score INT NULL, PRIMARY KEY (id), FOREIGN KEY(home_id) REFERENCES teams (id), FOREIGN KEY(away_id) REFERENCES teams (id), FOREIGN KEY(league_id) REFERENCES leagues (id))", False)

In [43]:
headers = {
    'X-RapidAPI-Key': api_football_key,
    'X-RapidAPI-Host': 'api-football-v1.p.rapidapi.com'
}

In [44]:
response = requests.get(f"https://api-football-v1.p.rapidapi.com/v3/fixtures?league={league_id}&season={season}", headers=headers)

In [45]:
response_parsed = json.loads(response.text)

In [65]:
def add_match_info_to_db(fixture):
    fixture_id, league_id, league_name, fixture_date, home_id, home_name, away_id, away_name, home_score, away_score = fixture['fixture']['id'], fixture['league']['id'], f"{fixture['league']['name']} ({fixture['league']['country']})", fixture['fixture']['timestamp'], fixture['teams']['home']['id'], fixture['teams']['home']['name'], fixture['teams']['away']['id'], fixture['teams']['away']['name'], fixture['goals']['home'], fixture['goals']['away']
    fixture_date_converted = datetime.fromtimestamp(fixture_date).strftime('%Y-%m-%d %H:%M:%S')
    execute_query(f"INSERT IGNORE INTO leagues (id, name) VALUES ({league_id}, '{league_name}')", False)
    execute_query(f"INSERT IGNORE INTO teams (id, name) VALUES ({home_id}, '{home_name}')", False)
    execute_query(f"INSERT IGNORE INTO teams (id, name) VALUES ({away_id}, '{away_name}')", False)
    execute_query(f"INSERT IGNORE INTO matches (id, date, league_id, home_id, away_id, home_score, away_score) VALUES ({fixture_id}, '{fixture_date_converted}', {league_id}, {home_id}, {away_id}, '{home_score}', '{away_score}')", False)
    print(f"{fixture['teams']['home']['name']} {fixture['goals']['home']} x {fixture['goals']['away']} {fixture['teams']['away']['name']}")

In [66]:
fixtures = [fixture for fixture in response_parsed['response'] if fixture['fixture']['timestamp'] < now]
for fixture in fixtures:
    add_match_info_to_db(fixture)

Atletico-MG 2 x 0 Internacional
Fluminense 0 x 0 Santos
Sao Paulo 4 x 0 Atletico Paranaense
Palmeiras 2 x 3 Ceara
Botafogo 1 x 3 Corinthians
Juventude 2 x 2 RB Bragantino
Fortaleza EC 0 x 1 Cuiaba
Atletico Goianiense 1 x 1 Flamengo
Avai 1 x 0 America Mineiro
Coritiba 3 x 0 Goias
America Mineiro 4 x 1 Juventude
Flamengo 3 x 1 Sao Paulo
RB Bragantino 4 x 0 Atletico Goianiense
Corinthians 3 x 0 Avai
Santos 2 x 1 Coritiba
Internacional 2 x 1 Fortaleza EC
Ceara 1 x 3 Botafogo
Goias 1 x 1 Palmeiras
Cuiaba 0 x 1 Fluminense
Atletico Paranaense 0 x 1 Atletico-MG
Atletico-MG 2 x 2 Coritiba
Fluminense 0 x 1 Internacional
RB Bragantino 1 x 1 Sao Paulo
Palmeiras 3 x 0 Corinthians
Santos 3 x 0 America Mineiro
Juventude 0 x 1 Cuiaba
Fortaleza EC 0 x 1 Ceara
Atletico Goianiense 1 x 1 Botafogo
Avai 3 x 2 Goias
Atletico Paranaense 1 x 0 Flamengo
America Mineiro 1 x 0 Atletico Paranaense
Flamengo 0 x 0 Palmeiras
Sao Paulo 2 x 1 Santos
Corinthians 1 x 0 Fortaleza EC
Botafogo 1 x 1 Juventude
Internacional 

In [80]:
fixtures_df = execute_query(f"SELECT m.id, m.date, l.name AS league, ht.name as home_team, at.name as away_team, m.home_score, m.away_score FROM matches AS m INNER JOIN teams AS ht ON (m.home_id = ht.id) INNER JOIN teams AS at ON (m.away_id = at.id) INNER JOIN leagues AS l ON (m.league_id = l.id) ORDER BY m.date DESC")

In [82]:
fixtures_df.head()

,id,date,league,home_team,away_team,home_score,away_score
0,838106,2022-06-14 21:30:00,Serie A (Brazil),Juventude,Santos,1,2
1,838095,2022-06-13 19:00:00,Serie A (Brazil),Botafogo,Avai,0,1
2,838097,2022-06-12 19:00:00,Serie A (Brazil),Fortaleza EC,Atletico Paranaense,0,0
3,838100,2022-06-12 18:00:00,Serie A (Brazil),Coritiba,Palmeiras,0,2
4,838093,2022-06-12 16:00:00,Serie A (Brazil),Sao Paulo,America Mineiro,1,0
